In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
import pandas as pd
import numpy as np
import json
meta_path = '/content/drive/MyDrive/Colab Notebooks/넷플릭스 영화추천/movies_metadata.csv'
meta = pd.read_csv(meta_path)

# genres : 영화 장르
# id(movieid) : 영화 고유번호, 사용자 평가 데이터와 연결할떄 인덱스 값으로 사용
# original title : 영화제목
meta.head()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [5]:
meta = meta[['id', 'original_title', 'original_language', 'genres']]

# id컬럼을 movieId컬럼으로 변경
meta = meta.rename(columns = {'id' : 'movieId'})

# 언어는 영어로 적힌것만
meta = meta[meta['original_language'] == 'en']
meta.head()

,movieId,original_title,original_language,genres
0,862,Toy Story,en,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '..."
1,8844,Jumanji,en,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '..."
2,15602,Grumpier Old Men,en,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ..."
3,31357,Waiting to Exhale,en,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam..."
4,11862,Father of the Bride Part II,en,"[{'id': 35, 'name': 'Comedy'}]"


In [6]:
# 크기 작은거 사용
ratings = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/넷플릭스 영화추천/ratings_small.csv')

# userId : 평가한 유저의 고유번호
# movieId : 영화 고유번호
# ratings : 해당 영화에 대한 유저의 평가
ratings.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
ratings = ratings[['userId', 'movieId', 'rating']]
ratings.head()

,userId,movieId,rating
0,1,31,2.5
1,1,1029,3.0
2,1,1061,3.0
3,1,1129,2.0
4,1,1172,4.0


In [8]:
ratings.describe()

,userId,movieId,rating
count,100004.000000,100004.000000,100004.000000
mean,347.011310,12548.664363,3.543608
std,195.163838,26369.198969,1.058064
min,1.000000,1.000000,0.500000
25%,182.000000,1028.000000,3.000000
50%,367.000000,2406.500000,4.000000
75%,520.000000,5418.000000,4.000000
max,671.000000,163949.000000,5.000000


# 데이터셋 정제

In [9]:
meta.dtypes

movieId              object
original_title       object
original_language    object
genres               object
dtype: object

In [10]:
meta.movieId = pd.to_numeric(meta.movieId, errors = 'coerce')
ratings.movieId = pd.to_numeric(ratings.movieId, errors = 'coerce')

In [11]:
meta.dtypes

movieId               int64
original_title       object
original_language    object
genres               object
dtype: object

In [15]:
meta.genres[0]

"[{'id': 16, 'name': 'Animation'}, {'id': 35, 'name': 'Comedy'}, {'id': 10751, 'name': 'Family'}]"

In [22]:
meta['genres'][0].replace('\'', '"')

'[{"id": 16, "name": "Animation"}, {"id": 35, "name": "Comedy"}, {"id": 10751, "name": "Family"}]'

In [16]:
def parse_genres(genres_str):
    genres = json.loads(genres_str.replace('\'', '"'))
    genres_list = []
    for g in genres:
        genres_list.append(g['name'])
    return genres_list

In [24]:
meta['genres'] =  meta['genres'].apply(parse_genres)

In [25]:
meta['genres']

0         [Animation, Comedy, Family]
1        [Adventure, Fantasy, Family]
2                   [Romance, Comedy]
3            [Comedy, Drama, Romance]
4                            [Comedy]
                     ...             
45459               [Science Fiction]
45460        [Drama, Action, Romance]
45463       [Action, Drama, Thriller]
45464                              []
45465                              []
Name: genres, Length: 32269, dtype: object

# 데이터셋 합치기

In [26]:
# movieId 기준으로 합치기
# Inner방식으로 합치기, , 같은 영화별 유저 평가 다 보여주기
data = pd.merge(ratings, meta, on = 'movieId', how = 'inner')

In [28]:
data.head()

,userId,movieId,rating,original_title,original_language,genres
0,1,1371,2.5,Rocky III,en,[Drama]
1,4,1371,4.0,Rocky III,en,[Drama]
2,7,1371,3.0,Rocky III,en,[Drama]
3,19,1371,4.0,Rocky III,en,[Drama]
4,21,1371,3.0,Rocky III,en,[Drama]


In [29]:
# 피벗테이블
matrix = data.pivot_table(index = 'userId', columns = 'original_title', values = 'rating')
matrix.head()

original_title,!Women Art Revolution,'Gator Bait,'Twas the Night Before Christmas,10 Items or Less,10 Things I Hate About You,"10,000 BC",11'09''01 - September 11,12 + 1,12 Angry Men,1408,...,Young and Innocent,Zaat,Zabriskie Point,Zapped Again!,Zardoz,Zodiac,eXistenZ,xXx,¡Three Amigos!,Мой сводный брат Франкенштейн
userId,,,,,,,,,,,,,,,,,,,,,
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [30]:
# 상관 관계 (피어슨)
G_WEIGHT = 0.1
def pearsonR(s1, s2):
    s1_c = s1 - s1.mean()
    s2_c = s2 - s2.mean()
    return np.sum(s1_c * s2_c) / np.sqrt(np.sum(s1_c ** 2) * np.sum(s2_c ** 2))

In [31]:
# 추천함수
# 영화 제목, 피벗테이블, 몇개 영화 추천해줌, 비슷한장르(similar_genre)
def recommend(input_movie, matrix, n, similar_genre=True):
    input_genres = meta[meta['original_title'] == input_movie]['genres'].iloc(0)[0]

    result = []

    for title in matrix.columns:
        # 영화제목이랑 , 입력받은 영화제목이 같은 경우 똑같은 영화를 추천하기떄문에 패스
        if title == input_movie:
            continue

        # 피어슨 상관계수 계산
        cor = pearsonR(matrix[input_movie], matrix[title])
        
        # 장르 비교
        if similar_genre and len(input_genres) > 0:
            temp_genres = meta[meta['original_title'] == title]['genres'].iloc(0)[0]

            # 입력받은 영화 장르랑 비교할 장르랑 몇개 일치하는지 확인
            # 2개일치하면[True, True, False] - > 2 
            same_count = np.sum(np.isin(input_genres, temp_genres))

            # 상관계수에 0.1 * 2해서 더해줌
            cor += (G_WEIGHT * same_count)
        
        if np.isnan(cor):
            continue
        else:
            # 빈리스트에 제목, 상관계수, 장르 추가
            result.append((title, '{:.2f}'.format(cor), temp_genres))
    # 내림차순 정렬
    result.sort(key=lambda r: r[1], reverse=True)

    return result[:n]

In [42]:
input_genres = meta[meta['original_title'] == 'Michael']['genres'].iloc(0)[0]
input_genres

['Comedy', 'Drama', 'Fantasy']

In [36]:
meta[meta['original_title'] == 'The Dark Knight']

,movieId,original_title,original_language,genres
12481,155,The Dark Knight,en,"[Drama, Action, Crime, Thriller]"
28700,72003,The Dark Knight,en,"[Action, Crime, Drama, Thriller]"


In [39]:
meta[meta['original_title'] == 'The Dark Knight']['genres']

12481    [Drama, Action, Crime, Thriller]
28700    [Action, Crime, Drama, Thriller]
Name: genres, dtype: object

In [40]:
meta[meta['original_title'] == 'The Dark Knight']['genres'].iloc(0)[0]

['Drama', 'Action', 'Crime', 'Thriller']

In [43]:
temp_genres =meta[meta['original_title'] == 'The Dark Knight']['genres'].iloc(0)[0]

In [45]:
np.sum(np.isin(input_genres, temp_genres))

1

In [46]:
recommend_result

[('About Schmidt', '0.40', ['Drama', 'Comedy']),
 ('Rocky V', '0.40', ['Drama']),
 ('54', '0.39', ['Music', 'Drama']),
 ('Michael', '0.39', ['Comedy', 'Drama', 'Fantasy']),
 ('Millennium', '0.39', ['Mystery', 'Drama', 'Science Fiction', 'Thriller']),
 ('Touch of Evil', '0.38', ['Drama', 'Thriller', 'Crime']),
 ('The House of the Spirits', '0.35', ['Romance', 'Drama']),
 ('Midnight in the Garden of Good and Evil',
  '0.33',
  ['Crime', 'Drama', 'Mystery', 'Thriller']),
 ('Population 436', '0.33', ['Drama', 'Horror', 'Mystery', 'Thriller']),
 ('Porgy and Bess', '0.33', ['Music', 'Drama', 'Romance'])]

In [32]:
recommend_result = recommend('Rocky III', matrix, 10, similar_genre=True)

pd.DataFrame(recommend_result, columns = ['Title', 'Correlation', 'Genre'])

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: RuntimeWarning: invalid value encountered in double_scalars
  


,Title,Correlation,Genre
0,About Schmidt,0.40,"[Drama, Comedy]"
1,Rocky V,0.40,[Drama]
2,54,0.39,"[Music, Drama]"
3,Michael,0.39,"[Comedy, Drama, Fantasy]"
4,Millennium,0.39,"[Mystery, Drama, Science Fiction, Thriller]"
5,Touch of Evil,0.38,"[Drama, Thriller, Crime]"
6,The House of the Spirits,0.35,"[Romance, Drama]"
7,Midnight in the Garden of Good and Evil,0.33,"[Crime, Drama, Mystery, Thriller]"
8,Population 436,0.33,"[Drama, Horror, Mystery, Thriller]"
9,Porgy and Bess,0.33,"[Music, Drama, Romance]"
